# Clustering river profiles for geomorphology

This is a tutorial on how to perform agglomerative hierarchical clustering on river profiles, using an example from the Pozo catchment, Santa Cruz Island.  We can use this to classify the landscape into different domains, which can be used to explore erosion rates, lithology, or the influence of climate and tectonics on landscapes.  You can find more details of the method in this paper:

Clubb, F.J., Bookhagen, B. and Rheinwalt, A. (2019) Clustering River Profiles to Classify Geomorphic Domains, _JGR: Earth Surface_, https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2019JF005025

You can find all the code in this GitHub repository. This repository is organised so that the code required to perform the clustering and plotting is stored in separate files, and the file `cluster-river-profiles.py` can be run as a command line wrapper. This notebook is an alternative to this wrapper script that walks through how to perform the clustering in a bit more detail.

**IMPORTANT**: The example data are too big to store on GitHub. The first step is to download these data and store it in the same folder as the code:
https://durhamuniversity-my.sharepoint.com/:u:/g/personal/bjdd72_durham_ac_uk/Ee-oVKcprrBHvaYWsVhjJ1MB0VLblXKib4bkEZV4wCIqxQ?e=TMsRqT

If you don't do this, then the rest of the notebook won't work!
We firstly extract the data from the compressed archive:

In [ ]:
import tarfile
tf = tarfile.open("example_data.tar.gz")
tf.extractall()

The next step is to import the required functions:

In [ ]:
# Firstly, import the required files and set matplotlib to show plots in the notebook.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import clustering as cl
import plotting as pl
import raster_plotting as rpl

After importing the scripts, we need to define a few arguments in the block below:
1. `data_dir`: The data directory. We'll use the `example_data` directory in this repository.
2. `fname`: The name of the DEM. In this example, we'll use the Pozo catchment, and the file is called `Pozo_DTM`
3. `step`: This is the spacing that you want the profiles to have for the clustering. This is dependent on the resolution of your DEM. Here we're using a 1 m DEM, so we'll use a 2 m spacing.
4. `so`: The stream order of the rivers that you wish to perform the clustering over. Here we'll focus on the first order streams, so we'll set `so = 1` 

In [ ]:
# User-defined variables
data_dir = './example_data/'
fname = 'Pozo_DTM'
step = 2
so = 1

We then need some additional variables which you should keep constant in all runs.

In [ ]:
# Additional variables
slope_window = 25    # the window size for calculating the slope
min_length = 5       # the minimum length of a stream for it to be included in the clustering (in  nodes)
method = 'ward'      # the method for clustering. See Scipy docs for more details: https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html


The code uses a `csv` file, in this case `Pozo_DTM_all_tribs.csv`, to perform the clustering. This csv file simply lists the latitude and longitude of every point in the river network, along with other variables such as drainage area and elevation. You can create this file however you want: I do it using LSDTopoTools[https://github.com/LSDtopotools/LSDTopoTools2], but you could also use TopoToolbox or your own Python code.  Just make sure it follows the same structure as the file provided in this repository.

We use `pandas` to read in the csv file to a dataframe, and then calculate the slope at each point along the river using a moving window approach. This will take quite a few minutes to run: it's the slowest part of the process. This function produces a figure showing the slope of all the profiles - we can see it's pretty messy!

In [ ]:
# read in the csv file with the river network
df = pd.read_csv(data_dir+fname+'_all_tribs.csv')
print("Got the csv of the channel network")
print(df)

After calculating the slope, we then get the profiles for the chosen stream order and step spacing:

We can make a plot of the profiles for this stream order - it looks pretty messy!

In [ ]:
# get the profiles for the chosen stream order
so_df = cl.GetProfilesByStreamOrder(data_dir, fname, df, step, slope_window, so)

# if stream order is greater than one, remove any non-unique profiles
if so > 1:
    so_df = cl.RemoveNonUniqueProfiles(so_df)

# remove any profiles shorter than the threshold length
so_df = cl.RemoveProfilesShorterThanThresholdLength(so_df, min_length)

We then perform the clustering. To do this we select a "threshold level", which helps to determine the number of clusters. To get the grouping with the most distinct clusters, we set the threshold level to 0:

In [ ]:
threshold_level = 0
# perform the clustering and save the output as a dataframe
cluster_df = cl.ClusterProfilesVaryingLength(data_dir, data_dir, fname, so_df, method, so, threshold_level)

The graph above shows the dendrogram of how all the rivers are related to each other. The threshold level is marked with the red line. You can try changing the `threshold_level` to 1 to see how this changes the cutting of the dendrogram and the number of clusters.

This function returns the river profiles as dataframe (`cluster_df`) which has the cluster ID assigned to each river. We can use this to do some more plotting. For example, we can plot the profiles separately by cluster:

In [ ]:
# plot the gradient vs. distance from source for each cluster
pl.PlotProfilesByCluster(data_dir, data_dir, fname, so)

We can aggregate these profiles by plotting the median profile in each cluster:

In [ ]:
# plot the median profiles by cluster
pl.PlotMedianProfiles(data_dir, data_dir, fname, so)

We can also make a plot of the hillshade of the catchment with the river network coloured by the cluster number:

In [ ]:
# make a hillshade plot of the raster with the clusters overlain
rpl.PlotHillshadewithClusters(data_dir, data_dir, fname, so)

In the case of the Pozo catchment, we can see that there seems to be a distinct grouping of the clusters with
1. Profiles in the red cluster near the outlet of the catchment
2. Profiles in the blue cluster near the headwaters

We can compare this to the geological map:

In [ ]:
geol_raster = 'Pozo_geology.tif'
rpl.PlotRasterLithologyWithClusters(data_dir, data_dir, fname, so, geol_raster)

We can see that the majority of profiles in the red cluster fall within the blue lithology, a weakly resistant shale, compared to the profiles in the blue cluster which fall within the orange-brown lithologies (more resistant volcaniclastics). In this case, the morphology of river profiles is dominated by lithology!